In [1]:
# !pip install peft transformers bitsandbytes datasets

## 1. Prepare Energy Domain Dataset (Q&A format)
Use scraped energy text to create instruction-tuning pairs.

In [2]:
import os
import json
# Ensure the data directory exists
os.makedirs('../../data', exist_ok=True)
# Example: create a small Q&A dataset
qa_data = [
    {"question": "What is oil production forecasting?", "answer": "It is the prediction of future oil output from wells using historical and engineering data."},
    {"question": "What factors affect oil production?", "answer": "Reservoir pressure, choke size, and weather are key factors."}
]
with open('../../data/energy_qa.json', 'w') as f:
    json.dump(qa_data, f)

## 2. Load Base Model and Tokenizer (4-bit QLoRA)
Use Hugging Face Transformers and peft.

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
import os
from huggingface_hub import login
from dotenv import load_dotenv

# Load environment variables from .env file in the project root (works in Jupyter)
load_dotenv(os.path.abspath(os.path.join(os.getcwd(), '../../.env')))

# Get Hugging Face token from environment variable
hf_token = os.getenv('HUGGINGFACE_TOKEN')
if hf_token is not None:
    login(hf_token)

# Use a small, fast model for LoRA fine-tuning
model_name = 'distilgpt2'  # switched from meta-llama/Llama-3.1-8B to distilgpt2 for speed

print(f"Using model: {model_name}")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token)
# Set padding token if missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=hf_token)


/Users/justin/miniconda3_Jun2023/envs/lora-llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using model: distilgpt2


/Users/justin/miniconda3_Jun2023/envs/lora-llm/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:1041: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/Users/justin/miniconda3_Jun2023/envs/lora-llm/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


## 3. Configure LoRA
Use r=16, lora_alpha=32, target_modules=["q_proj", "v_proj"] (see lora_config.yaml).

In [4]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # Llama compatible modules
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
 )
model = get_peft_model(model, lora_config)

ValueError: Target modules {'v_proj', 'q_proj'} not found in the base model. Please check the target modules and try again.

## 4. Fine-tune Model
Train on the Q&A dataset.

In [ ]:
from datasets import load_dataset
from transformers import TrainingArguments, Trainer
def preprocess(example):
    return tokenizer(example['question'] + '\n' + example['answer'], truncation=True, padding='max_length', max_length=512)
dataset = load_dataset('json', data_files='../../data/energy_qa.json')
tokenized = dataset.map(preprocess)
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    num_train_epochs=1,
    fp16=False,  # Disable fp16 for CPU or unsupported devices
    output_dir="./lora_output",
    logging_steps=10,
    save_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['train'],
)
trainer.train()
model.save_pretrained('./lora_output')

## 5. Evaluate Model
Evaluate on perplexity and domain-specific accuracy.

In [ ]:
# Example: compute perplexity
import torch
import math
def compute_perplexity(model, tokenizer, text):
    enc = tokenizer(text, return_tensors='pt')
    input_ids = enc['input_ids']
    with torch.no_grad():
        loss = model(input_ids, labels=input_ids).loss
    return math.exp(loss.item())
compute_perplexity(model, tokenizer, 'What is oil production forecasting?')

## 6. Model Card
See [MODEL_CARD.md](../models/energy_llm/MODEL_CARD.md) for details.

## 7. Deploy LoRA Adapters to Azure ML
Deploy LoRA adapters for inference. See deployment script.

In [ ]:
# See models/energy_llm/inference.py for adapter loading
# !python ../../models/energy_llm/inference.py

## Environment Setup Instructions

To run your notebooks reliably, use two separate environments:


**1. LoRA Fine-Tuning (LLM) Environment**

- Create with: `conda env create -f environment.lora-llm.yml`

- Activate with: `conda activate lora-llm`

- Use for: LoRA fine-tuning, LLM workflows



**2. PyTorch Forecasting Environment**

- Create with: `conda env create -f environment.forecasting.yml`

- Activate with: `conda activate forecasting`

- Use for: PyTorch Forecasting workflows



**How to use:**

- Before running a notebook, activate the matching environment in your terminal or Jupyter kernel selector.
- This ensures all dependencies are compatible and avoids version conflicts.


You can find both environment files in the project root.